### Digit Recognition Notebook
    A jupyter notebook explaining how the above Python script works and discussing its performance.

## Introduction
- This notebook contains information and explanations about a small subsection of object recognition—digit recognition. Using TensorFlow, an open-source Python library developed by the Google Brain labs for deep learning research, it will take hand-drawn images of the numbers 0-9 and build and train a neural network to recognize and predict the correct label for the digit displayed.

## Step 1 - Configuring the Project
- Before recognition program can be developed, a few dependencies need to be installed.
- By opening the file in text editor dependencies can be defined by adding the following lines to specify the Image, NumPy, and TensorFlow libraries and their versions:
    - image==1.5.20
    - numpy==1.14.3
    - tensorflow==1.4.0
- Once the file is saved and editor closed libraries can be installed with the following command:
    - $ pip install -r requirements.txt

## Step 2 - Importing the MNIST Dataset
- The dataset that was used in this program is called the MNIST dataset, and it is a classic in the machine learning community. This dataset is made up of images of handwritten digits, 28x28 pixels in size. Here are some examples of the digits included in the dataset:
<img src="Images/Sample MNIST.png" alt="Sample MNIST" title="Sample MNIST" />

- Following imports were required: 

In [ ]:
# Import the TensorFlow library
import tensorflow as tf

# Import the MNIST dataset and store the image data in the variable mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) # y labels are oh-encoded

- When reading in the data, we are using one-hot-encoding to represent the labels (the actual digit drawn, e.g. "3") of the images. One-hot-encoding uses a vector of binary values to represent numeric or categorical values. As our labels are for the digits 0-9, the vector contains ten values, one for each possible digit. One of these values is set to 1, to represent the digit at that index of the vector, and the rest are set to 0. For example, the digit 3 is represented using the vector [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]. As the value at index 3 is stored as 1, the vector therefore represents the digit 3.

- To represent the actual images themselves, the 28x28 pixels are flattened into a 1D vector which is 784 pixels in size. Each of the 784 pixels making up the image is stored as a value between 0 and 255. This determines the grayscale of the pixel, as our images are presented in black and white only. So a black pixel is represented by 255, and a white pixel by 0, with the various shades of gray somewhere in between. 

## Step 3 - Defining the Neural Network Architecture
- The architecture of the neural network refers to elements such as the number of layers in the network, the number of units in each layer, and how the units are connected between layers. As neural networks are loosely inspired by the workings of the human brain, here the term unit is used to represent what we would biologically think of as a neuron. Like neurons passing signals around the brain, units take some values from previous units as input, perform a computation, and then pass on the new value as output to other units. These units are layered to form the network, starting at a minimum with one layer for inputting values, and one layer to output values. The term hidden layer is used for all of the layers in between the input and output layers, i.e. those "hidden" from the real world.

- Different architectures can yield drastically different results, as the performance can be thought of as a function of the architecture among other things, such as the parameters, the data, and the duration of training. 

In [ ]:
n_input = 784   # input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10   # output layer (0-9 digits)

- The following diagram shows a visualization of the architecture that was designed, with each layer fully connected to the surrounding layers:
<img src="Images/Fig1.png" alt="visualization of the architecture" title="visualization of the architecture" />
- The term "deep neural network" relates to the number of hidden layers, with "shallow" usually meaning just one hidden layer, and "deep" referring to multiple hidden layers. Given enough training data, a shallow neural network with a sufficient number of units should theoretically be able to represent any function that a deep neural network can. But it is often more computationally efficient to use a smaller deep neural network to achieve the same task that would require a shallow network with exponentially more hidden units. Shallow neural networks also often encounter overfitting, where the network essentially memorizes the training data that it has seen, and is not able to generalize the knowledge to new data. This is why deep neural networks are more commonly used: the multiple layers between the raw input data and the output label allow the network to learn features at various levels of abstraction, making the network itself better able to generalize.

- Other elements of the neural network that need to be defined here are the hyperparameters. Unlike the parameters that will get updated during training, these values are set initially and remain constant throughout the process. 

In [ ]:
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

- The learning rate represents ow much the parameters will adjust at each step of the learning process. These adjustments are a key component of training: after each pass through the network we tune the weights slightly to try and reduce the loss. Larger learning rates can converge faster, but also have the potential to overshoot the optimal values as they are updated. The number of iterations refers to how many times we go through the training step, and the batch size refers to how many training examples we are using at each step. The dropout variable represents a threshold at which we elimanate some units at random. We will be using dropout in our final hidden layer to give each unit a 50% chance of being eliminated at every training step. This helps prevent overfitting.

## Step 4 - Building the TensorFlow Graph
- To build network, needed to set up the network as a computational graph for TensorFlow to execute. The core concept of TensorFlow is the tensor, a data structure similar to an array or list. initialized, manipulated as they are passed through the graph, and updated through the learning process. 

In [ ]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

- The only parameter that needs to be specified at its declaration is the size of the data that will be feed in. For X used a shape of [None, 784], where None represents any amount, as it will be feeded in an undefined number of 784-pixel images. The shape of Y is [None, 10] as it will be used for an undefined number of label outputs, with 10 possible classes. The keep_prob tensor is used to control the dropout rate, and we initialize it as a placeholder rather than an immutable variable to use the same tensor both for training (when dropout is set to 0.5) and testing (when dropout is set to 1.0).

- The parameters that the network will update in the training process are the weight and bias values, so for these need to set an initial value rather than an empty placeholder. These values are essentially where the network does its learning, as they are used in the activation functions of the neurons, representing the strength of the connections between units.

- Since the values are optimized during training, could set them to zero. But the initial value actually has a significant impact on the final accuracy of the model. Random values were used from a truncated normal distribution for the weights. As they required to be close to zero, so they can adjust in either a positive or negative direction, and slightly different, so they generate different errors. This will ensure that the model learns something useful. 

In [ ]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}

- For the bias, used a small constant value to ensure that the tensors activate in the intial stages and therefore contribute to the propagation. The weights and bias tensors are stored in dictionary objects for ease of access.

In [ ]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

- Next, set up the layers of the network by defining the operations that will manipulate the tensors.

In [ ]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

- Each hidden layer will execute matrix multiplication on the previous layer’s outputs and the current layer’s weights, and add the bias to these values. At the last hidden layer, it applies a dropout operation using keep_prob value of 0.5.

- The final step in building the graph is to define the loss function that needs to be optimized. A popular choice of loss function in TensorFlow programs is cross-entropy, also known as log-loss, which quantifies the difference between two probability distributions (the predictions and the labels). A perfect classification would result in a cross-entropy of 0, with the loss completely minimized.

- A process named gradient descent optimization is a common method for finding the (local) minimum of a function by taking iterative steps along the gradient in a negative (descending) direction. There are several choices of gradient descent optimization algorithms already implemented in TensorFlow, and decided to use the Adam optimizer. This extends upon gradient descent optimization by using momentum to speed up the process through computing an exponentially weighted average of the gradients and using that in the adjustments.

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

## Step 5 - Training and Testing
- The training process involves feeding the training dataset through the graph and optimizing the loss function. Every time the network iterates through a batch of more training images, it updates the parameters to reduce the loss in order to more accurately predict the digits shown.

- Before starting the training process, methods need to be defined of evaluating the accuracy so results can print it out on mini-batches of data while its trained. These printed statements will allow to check that from the first iteration to the last, loss decreases and accuracy increases; they will also allow to track whether or not have ran enough iterations to reach a consistent and optimal result.

In [ ]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

- In correct_pred, used the arg_max function to compare which images are being predicted correctly by looking at the output_layer (predictions) and Y (labels), and used the equal function to return this as a list of [Booleans](tps://www.digitalocean.com/community/tutorials/understanding-data-types-in-python-3#booleans). Then cast this list to floats and calculated the mean to get a total accuracy score. 

- Initialized a session for running the graph. In this session feeded the network with training examples, and once it was trained, feeded the same graph with new test examples to determine the accuracy of the model.

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

- The essence of the training process in deep learning is to optimize the loss function. Here the aim was to minimize the difference between the predicted labels of the images, and the true labels of the images. The process involves four steps which are repeated for a set number of iterations:
    - Propagate values forward through the network
    - Compute the loss
    - Propagate values backward through the network
    - Update the parameters

- At each training step, the parameters are adjusted slightly to try and reduce the loss for the next step. As the learning progresses, reduction is seen in loss, and eventually training can stop and use the network as a model for testing new data. 

In [ ]:
# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})

    # print loss and accuracy (per minibatch)
    if i%100==0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
        print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

- After 100 iterations of each training step in which a mini-batch of images through the network is feeded, the loss and accuracy of that batch is printed out.

- Once the training is complete, session can be ran on the test images. This time keep_prob dropout rate of 1.0 is being used to ensure all units are active in the testing process.

In [ ]:
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
print("\nAccuracy on test set:", test_accuracy)

## Conclusion
- In this program I managed to successfully train a neural network to classify the MNIST dataset with around 92% accuracy and tested it on sample images that I have created in Paint. Current state-of-the-art research achieves around 99% on this same problem, using more complex network architectures involving convolutional layers, the ones I have described in mnist-dataset notebook. These use the 2D structure of the image to better represent the contents, unlike this method which flattened all the pixels into one vector of 784 units.

###### References: 
- __[Information](https://www.digitalocean.com/community/tutorials/how-to-build-a-neural-network-to-recognize-handwritten-digits-with-tensorflow)__